# Integration Patterns - Multi-Provider & Modern Token Management

This notebook focuses on practical patterns for production AI applications:

1. **Multiple LLM Providers** - Using OpenAI and Anthropic together
2. **Provider Fallback** - Backup strategies when one provider fails
3. **Modern Token Management** - Direct API usage tracking & streaming with progressive monitoring
4. **External Data Enrichment** - Combining APIs with LLM prompts
5. **Production Configuration** - Environment-based settings

We'll use OpenAI and Anthropic APIs throughout, focusing on **modern token management** using:
- **Direct API usage tracking** - Using exact token counts from API responses
- **Streaming with progressive tracking** - Real-time token monitoring during streaming

## Learning Goals

After this lesson, you will be able to:

- [ ] Use multiple LLM providers (OpenAI, Anthropic) in the same application
- [ ] Pass outputs from one LLM to another LLM
- [ ] Implement provider fallback strategies
- [ ] Track token usage using **direct API response data** (most reliable method)
- [ ] Monitor token consumption in **real-time during streaming**
- [ ] Enrich prompts with external data sources
- [ ] Apply production configuration patterns

## Setup

Let's import the necessary libraries and set up our clients.

!pip install anthropic

In [ ]:
import os
import time
from openai import OpenAI
from anthropic import Anthropic
from typing import Optional, Dict, Any, Iterator
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize clients
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# Default models
OPENAI_MODEL = "gpt-4o-mini"
ANTHROPIC_MODEL = "claude-3-5-sonnet-20241022"

print("✅ Setup complete!")

---

## 1. Working with Multiple LLM Providers

### Why Use Multiple Providers?

Different LLM providers have different strengths:
- **OpenAI (GPT-4o, GPT-4o-mini)**: Great for general tasks, fast, good structured output
- **Anthropic (Claude)**: Excellent for long context, detailed analysis, safety-focused

Let's set up helper functions for both providers.

In [ ]:
def ask_openai(prompt: str, model: str = OPENAI_MODEL) -> Dict[str, Any]:
    """
    Ask OpenAI a question and return response with usage data.
    
    Args:
        prompt: Question or instruction
        model: Model to use
    
    Returns:
        Dictionary with response text and usage information
    """
    response = openai_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Extract usage data directly from API response
    usage = response.usage
    
    return {
        "response": response.choices[0].message.content,
        "usage": {
            "prompt_tokens": usage.prompt_tokens,
            "completion_tokens": usage.completion_tokens,
            "total_tokens": usage.total_tokens
        }
    }

def ask_anthropic(prompt: str, model: str = ANTHROPIC_MODEL) -> Dict[str, Any]:
    """
    Ask Anthropic Claude a question and return response with usage data.
    
    Args:
        prompt: Question or instruction
        model: Model to use
    
    Returns:
        Dictionary with response text and usage information
    """
    response = anthropic_client.messages.create(
        model=model,
        max_tokens=1024,
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Extract usage data directly from API response
    usage = response.usage
    
    return {
        "response": response.content[0].text,
        "usage": {
            "input_tokens": usage.input_tokens,
            "output_tokens": usage.output_tokens
        }
    }

print("✅ Provider functions ready! (now with direct API usage tracking)")

### Comparing Providers

Let's compare responses from both providers:

In [ ]:
def compare_providers(prompt: str) -> Dict[str, Any]:
    """
    Ask the same question to both providers and compare responses + token usage.
    
    Args:
        prompt: Question to ask
    
    Returns:
        Dictionary with both responses and usage data
    """
    print(f"Question: {prompt}\n")
    
    # Ask OpenAI
    print("Asking OpenAI...")
    openai_result = ask_openai(prompt)
    print(f"OpenAI Response: {openai_result['response']}")
    print(f"OpenAI Usage: {openai_result['usage']}\n")
    
    # Ask Anthropic
    print("Asking Anthropic...")
    anthropic_result = ask_anthropic(prompt)
    print(f"Anthropic Response: {anthropic_result['response']}")
    print(f"Anthropic Usage: {anthropic_result['usage']}\n")
    
    return {
        "openai": openai_result,
        "anthropic": anthropic_result
    }

# Test with a simple prompt
# results = compare_providers("Explain machine learning in one sentence")
# print("✅ Comparison complete!")

### Passing Outputs Between Providers

One powerful pattern: use one LLM's output as another's input.

In [ ]:
def generate_and_review(topic: str) -> Dict[str, Any]:
    """
    Use OpenAI to generate content, then Anthropic to review it.
    Tracks token usage from both API calls.
    
    Args:
        topic: Topic to write about
    
    Returns:
        Dictionary with content, review, and usage data
    """
    # Step 1: Generate content with OpenAI
    print(f"Step 1: Generating content about '{topic}' with OpenAI...")
    content_prompt = f"Write a short paragraph about {topic}"
    openai_result = ask_openai(content_prompt)
    content = openai_result['response']
    print(f"Content: {content}")
    print(f"OpenAI Usage: {openai_result['usage']}\n")
    
    # Step 2: Review with Anthropic
    print("Step 2: Reviewing with Anthropic...")
    review_prompt = f"""Review this paragraph and provide feedback:

{content}

Provide:
1. What's good about it
2. What could be improved
3. A rating out of 10"""
    
    anthropic_result = ask_anthropic(review_prompt)
    review = anthropic_result['response']
    print(f"Review: {review}")
    print(f"Anthropic Usage: {anthropic_result['usage']}\n")
    
    return {
        "topic": topic,
        "content": content,
        "review": review,
        "usage": {
            "openai": openai_result['usage'],
            "anthropic": anthropic_result['usage']
        }
    }

# Test (commented out to avoid API calls)
# result = generate_and_review("artificial intelligence")
# print("✅ Generate and review complete!")

---

## 2. Provider Fallback Patterns

What happens when one provider fails or is rate-limited? You need a backup.

In [ ]:
def ask_with_fallback(prompt: str, primary: str = "openai", secondary: str = "anthropic") -> Dict[str, Any]:
    """
    Try primary provider first, fall back to secondary if it fails.
    Returns usage data from whichever provider succeeded.
    
    Args:
        prompt: Question to ask
        primary: Primary provider to try first
        secondary: Backup provider
    
    Returns:
        Response and usage data from whichever provider succeeded
    """
    # Try primary provider
    try:
        if primary == "openai":
            print("Trying OpenAI (primary)...")
            result = ask_openai(prompt)
            print("✓ OpenAI succeeded")
            return {
                "provider": "openai",
                "response": result["response"],
                "usage": result["usage"]
            }
    
    except Exception as e:
        print(f"✗ OpenAI failed: {e}")
        print(f"Falling back to {secondary}...")
    
    # If primary failed, try secondary
    try:
        if secondary == "anthropic":
            print("Trying Anthropic (fallback)...")
            result = ask_anthropic(prompt)
            print("✓ Anthropic succeeded")
            return {
                "provider": "anthropic",
                "response": result["response"],
                "usage": result["usage"]
            }
    
    except Exception as e:
        print(f"✗ Anthropic also failed: {e}")
        raise Exception("All providers failed")

# Test (commented out)
# result = ask_with_fallback("What is machine learning?")
# print(f"\nUsed provider: {result['provider']}")
# print(f"Response: {result['response'][:100]}...")
# print(f"Usage: {result['usage']}")

---

## 3. Modern Token Management

Modern token management uses **direct API usage tracking** - the most reliable method. Instead of estimating tokens before API calls, we use the exact token counts returned by the API responses. This is more accurate and eliminates estimation errors.

We'll also explore **streaming with progressive tracking** to monitor token consumption in real-time.

### Direct API Usage Tracking

**Key Principle**: Use the exact token counts from API responses, not estimates.

Both OpenAI and Anthropic APIs return usage information in their responses. This is the most reliable method because:
1. **Accuracy**: The API knows exactly how many tokens were used
2. **No estimation errors**: No need to guess or approximate
3. **Model-specific**: Works correctly for all models without special handling

In [ ]:
# Example: Direct API usage tracking
# When you make an API call, the response includes exact token counts

example_prompt = "Explain machine learning in one sentence"

print("Example: Getting exact token counts from API response")
print("=" * 60)

# OpenAI returns usage in response.usage
print("\nOpenAI API Response Structure:")
print("- response.usage.prompt_tokens    (input tokens)")
print("- response.usage.completion_tokens (output tokens)")
print("- response.usage.total_tokens     (total)")

# Anthropic returns usage in response.usage
print("\nAnthropic API Response Structure:")
print("- response.usage.input_tokens     (input tokens)")
print("- response.usage.output_tokens   (output tokens)")

print("\n✅ No estimation needed - use the exact counts from API responses!")

### Token Budget Manager

Now let's create a token budget manager that tracks usage across providers:

In [ ]:
class TokenBudgetManager:
    """Manage token budgets using direct API usage data."""
    
    def __init__(self, daily_budget: float = 100.0):
        """
        Initialize token budget manager.
        
        Args:
            daily_budget: Daily budget in dollars
        """
        self.daily_budget = daily_budget
        self.used_budget = 0.0
        self.provider_usage = {
            "openai": {"input_tokens": 0, "output_tokens": 0, "cost": 0.0},
            "anthropic": {"input_tokens": 0, "output_tokens": 0, "cost": 0.0}
        }
        
        # Pricing per 1M tokens (as of 2026-01)
        self.pricing = {
            "openai": {
                "gpt-4o-mini": {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
                "gpt-4o": {"input": 2.50 / 1_000_000, "output": 10.00 / 1_000_000}
            },
            "anthropic": {
                "claude-3-5-sonnet-20241022": {"input": 3.00 / 1_000_000, "output": 15.00 / 1_000_000}
            }
        }
    
    def calculate_cost(self, provider: str, model: str, input_tokens: int, output_tokens: int) -> float:
        """
        Calculate cost for a request using exact token counts.
        
        Args:
            provider: Provider name ("openai" or "anthropic")
            model: Model name
            input_tokens: Number of input tokens (from API response)
            output_tokens: Number of output tokens (from API response)
        
        Returns:
            Cost in dollars
        """
        if provider not in self.pricing:
            return 0.0
        
        if model not in self.pricing[provider]:
            return 0.0
        
        pricing = self.pricing[provider][model]
        input_cost = input_tokens * pricing["input"]
        output_cost = output_tokens * pricing["output"]
        return input_cost + output_cost
    
    def track_request(self, provider: str, model: str, input_tokens: int, output_tokens: int) -> Dict[str, Any]:
        """
        Track a request using exact token counts from API response.
        
        Args:
            provider: Provider name
            model: Model name
            input_tokens: Exact input tokens from API response
            output_tokens: Exact output tokens from API response
        
        Returns:
            Dictionary with cost and budget status
        """
        cost = self.calculate_cost(provider, model, input_tokens, output_tokens)
        
        # Check if we're over budget
        if self.used_budget + cost > self.daily_budget:
            remaining = self.daily_budget - self.used_budget
            raise Exception(
                f"Token budget exceeded! "
                f"Request cost ${cost:.6f}, but only ${remaining:.6f} remaining. "
                f"Daily budget: ${self.daily_budget:.2f}"
            )
        
        # Update usage with exact counts
        self.used_budget += cost
        if provider in self.provider_usage:
            self.provider_usage[provider]["input_tokens"] += input_tokens
            self.provider_usage[provider]["output_tokens"] += output_tokens
            self.provider_usage[provider]["cost"] += cost
        
        return {
            "cost": cost,
            "remaining_budget": self.daily_budget - self.used_budget,
            "budget_used_percent": (self.used_budget / self.daily_budget) * 100
        }
    
    def get_summary(self) -> Dict[str, Any]:
        """Get budget summary."""
        return {
            "daily_budget": self.daily_budget,
            "used_budget": self.used_budget,
            "remaining_budget": self.daily_budget - self.used_budget,
            "budget_used_percent": (self.used_budget / self.daily_budget) * 100,
            "provider_usage": self.provider_usage
        }
    
    def print_summary(self):
        """Print budget summary."""
        summary = self.get_summary()
        print("\n" + "=" * 60)
        print("TOKEN BUDGET SUMMARY")
        print("=" * 60)
        print(f"Daily Budget: ${summary['daily_budget']:.2f}")
        print(f"Used: ${summary['used_budget']:.4f} ({summary['budget_used_percent']:.1f}%)")
        print(f"Remaining: ${summary['remaining_budget']:.4f}")
        print("\nProvider Breakdown:")
        for provider, usage in summary['provider_usage'].items():
            if usage['input_tokens'] > 0 or usage['output_tokens'] > 0:
                print(f"\n  {provider.upper()}:")
                print(f"    Input tokens: {usage['input_tokens']:,}")
                print(f"    Output tokens: {usage['output_tokens']:,}")
                print(f"    Total cost: ${usage['cost']:.4f}")
        print("=" * 60)

# Create a budget manager
budget_manager = TokenBudgetManager(daily_budget=10.0)
print("✅ Token budget manager ready!")

In [ ]:
def ask_openai_with_budget(
    prompt: str,
    model: str = OPENAI_MODEL,
    budget_manager: TokenBudgetManager = None
) -> Dict[str, Any]:
    """
    Ask OpenAI with token budget tracking using DIRECT API usage data.
    
    Modern approach: Use exact token counts from API response, not estimates.
    
    Args:
        prompt: Question to ask
        model: Model to use
        budget_manager: TokenBudgetManager instance
    
    Returns:
        Dictionary with response and budget info
    """
    # Make API call - get response with usage data
    result = ask_openai(prompt, model)
    
    # Extract exact token counts from API response
    input_tokens = result['usage']['prompt_tokens']
    output_tokens = result['usage']['completion_tokens']
    
    # Track with budget manager using exact counts
    if budget_manager:
        budget_info = budget_manager.track_request(
            provider="openai",
            model=model,
            input_tokens=input_tokens,
            output_tokens=output_tokens
        )
        print(f"💰 Tokens: {input_tokens} input + {output_tokens} output = {result['usage']['total_tokens']} total")
        print(f"   Cost: ${budget_info['cost']:.6f} (${budget_info['remaining_budget']:.4f} remaining)")
    
    return {
        "response": result['response'],
        "usage": result['usage'],
        "provider": "openai",
        "model": model
    }

def ask_anthropic_with_budget(
    prompt: str,
    model: str = ANTHROPIC_MODEL,
    budget_manager: TokenBudgetManager = None
) -> Dict[str, Any]:
    """
    Ask Anthropic with token budget tracking using DIRECT API usage data.
    
    Modern approach: Use exact token counts from API response, not estimates.
    
    Args:
        prompt: Question to ask
        model: Model to use
        budget_manager: TokenBudgetManager instance
    
    Returns:
        Dictionary with response and budget info
    """
    # Make API call - get response with usage data
    result = ask_anthropic(prompt, model)
    
    # Extract exact token counts from API response
    input_tokens = result['usage']['input_tokens']
    output_tokens = result['usage']['output_tokens']
    
    # Track with budget manager using exact counts
    if budget_manager:
        budget_info = budget_manager.track_request(
            provider="anthropic",
            model=model,
            input_tokens=input_tokens,
            output_tokens=output_tokens
        )
        print(f"💰 Tokens: {input_tokens} input + {output_tokens} output")
        print(f"   Cost: ${budget_info['cost']:.6f} (${budget_info['remaining_budget']:.4f} remaining)")
    
    return {
        "response": result['response'],
        "usage": result['usage'],
        "provider": "anthropic",
        "model": model
    }

print("✅ Budget-aware API functions ready! (using direct API usage tracking)")

### Streaming with Progressive Token Tracking

**Key Principle**: Monitor token consumption in real-time during streaming to cut off requests that exceed budgets before they finish.

When streaming responses, you can track tokens progressively as they arrive, allowing you to:
1. Monitor consumption in real-time
2. Stop requests early if they exceed budget
3. Provide better user experience with streaming

In [ ]:
def ask_openai_streaming(
    prompt: str,
    model: str = OPENAI_MODEL,
    budget_manager: TokenBudgetManager = None,
    max_output_tokens: Optional[int] = None
) -> Iterator[Dict[str, Any]]:
    """
    Stream OpenAI response with progressive token tracking.
    
    Args:
        prompt: Question to ask
        model: Model to use
        budget_manager: TokenBudgetManager instance
        max_output_tokens: Optional limit on output tokens
    
    Yields:
        Dictionary with chunk text and current usage stats
    """
    # Create streaming request
    stream = openai_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )
    
    full_response = ""
    input_tokens = None
    output_tokens = 0
    
    for chunk in stream:
        # Extract delta content
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            
            # Track output tokens progressively (approximate: ~4 chars per token)
            # Note: For exact counts, wait for final usage data
            output_tokens_approx = len(full_response) // 4
            
            # Check budget during streaming
            if budget_manager and input_tokens is not None:
                estimated_cost = budget_manager.calculate_cost(
                    "openai", model, input_tokens, output_tokens_approx
                )
                if budget_manager.used_budget + estimated_cost > budget_manager.daily_budget:
                    print(f"\n⚠️  Budget limit approaching! Stopping stream.")
                    break
            
            # Check max output tokens
            if max_output_tokens and output_tokens_approx >= max_output_tokens:
                print(f"\n⚠️  Max output tokens reached ({max_output_tokens}). Stopping stream.")
                break
        
        # Get usage data when available (in final chunk)
        if chunk.usage:
            input_tokens = chunk.usage.prompt_tokens
            output_tokens = chunk.usage.completion_tokens
            
            # Final budget tracking with exact counts
            if budget_manager:
                budget_info = budget_manager.track_request(
                    provider="openai",
                    model=model,
                    input_tokens=input_tokens,
                    output_tokens=output_tokens
                )
                yield {
                    "type": "usage",
                    "usage": {
                        "prompt_tokens": input_tokens,
                        "completion_tokens": output_tokens,
                        "total_tokens": chunk.usage.total_tokens
                    },
                    "budget_info": budget_info
                }
        
        # Yield content chunk
        if chunk.choices[0].delta.content:
            yield {
                "type": "content",
                "content": chunk.choices[0].delta.content,
                "full_response": full_response,
                "estimated_tokens": output_tokens_approx if 'output_tokens_approx' in locals() else 0
            }

def ask_anthropic_streaming(
    prompt: str,
    model: str = ANTHROPIC_MODEL,
    budget_manager: TokenBudgetManager = None,
    max_output_tokens: Optional[int] = None
) -> Iterator[Dict[str, Any]]:
    """
    Stream Anthropic response with progressive token tracking.
    
    Args:
        prompt: Question to ask
        model: Model to use
        budget_manager: TokenBudgetManager instance
        max_output_tokens: Optional limit on output tokens
    
    Yields:
        Dictionary with chunk text and current usage stats
    """
    # Create streaming request
    with anthropic_client.messages.stream(
        model=model,
        max_tokens=1024,
        messages=[{"role": "user", "content": prompt}]
    ) as stream:
        full_response = ""
        input_tokens = None
        output_tokens = 0
        
        for text_event in stream.text_stream:
            if text_event:
                full_response += text_event
                
                # Track output tokens progressively (approximate)
                output_tokens_approx = len(full_response) // 4
                
                # Check budget during streaming
                if budget_manager and input_tokens is not None:
                    estimated_cost = budget_manager.calculate_cost(
                        "anthropic", model, input_tokens, output_tokens_approx
                    )
                    if budget_manager.used_budget + estimated_cost > budget_manager.daily_budget:
                        print(f"\n⚠️  Budget limit approaching! Stopping stream.")
                        break
                
                # Check max output tokens
                if max_output_tokens and output_tokens_approx >= max_output_tokens:
                    print(f"\n⚠️  Max output tokens reached ({max_output_tokens}). Stopping stream.")
                    break
                
                yield {
                    "type": "content",
                    "content": text_event,
                    "full_response": full_response,
                    "estimated_tokens": output_tokens_approx
                }
        
        # Get final usage data
        message = stream.get_final_message()
        if message.usage:
            input_tokens = message.usage.input_tokens
            output_tokens = message.usage.output_tokens
            
            # Final budget tracking with exact counts
            if budget_manager:
                budget_info = budget_manager.track_request(
                    provider="anthropic",
                    model=model,
                    input_tokens=input_tokens,
                    output_tokens=output_tokens
                )
                yield {
                    "type": "usage",
                    "usage": {
                        "input_tokens": input_tokens,
                        "output_tokens": output_tokens
                    },
                    "budget_info": budget_info
                }

print("✅ Streaming functions with progressive tracking ready!")

# Example usage (commented out):
# print("\nExample: Streaming with progressive tracking")
# print("=" * 60)
# for chunk in ask_openai_streaming("Write a short story about AI", budget_manager=budget_manager):
#     if chunk["type"] == "content":
#         print(chunk["content"], end="", flush=True)
#     elif chunk["type"] == "usage":
#         print(f"\n\n📊 Final Usage: {chunk['usage']}")
#         print(f"💰 Budget: {chunk['budget_info']}")

In [ ]:
# Create a test budget manager with a small budget
test_budget = TokenBudgetManager(daily_budget=0.0010)  # $0.10 for testing

print("Testing token budget system:")
print("=" * 60)

# Test 1: Make a request within budget
try:
    print("\n--- Test 1: Request within budget ---")
    result = ask_openai_with_budget(
        "Say hello in one sentence.",
        model="gpt-4o-mini",
        budget_manager=test_budget
    )
    print(f"Response: {result['response']}")
    print(f"Tokens: {result['input_tokens']} input + {result['output_tokens']} output")
except Exception as e:
    print(f"Error: {e}")

# Show budget status
test_budget.print_summary()

# Test 2: Try to exceed budget
print("\n--- Test 2: Attempting to exceed budget ---")
try:
    # Try to make many requests to exceed budget
    for i in range(100):
        result = ask_openai_with_budget(
            f"Count to {i}",
            model="gpt-4o-mini",
            budget_manager=test_budget
        )
        print(f"Request {i+1} succeeded")
except Exception as e:
    print(f"Budget limit reached: {e}")

test_budget.print_summary()

---

## 4. Enriching Prompts with External Data

LLMs have knowledge cutoffs. Fetch external data and add it to prompts for real-time information.

In [ ]:
import requests
from typing import Optional

def fetch_weather_data(city: str, api_key: Optional[str] = None) -> Optional[Dict[str, Any]]:
    """
    Fetch current weather data for a city (mock implementation).
    
    Args:
        city: City name
        api_key: API key (optional for demo)
    
    Returns:
        Weather data dictionary or None
    """
    # This is a mock implementation
    # In production, you'd use a real weather API like OpenWeatherMap
    
    # Mock data for demonstration
    mock_weather = {
        "city": city,
        "temperature": 22.5,
        "feels_like": 21.0,
        "description": "partly cloudy",
        "humidity": 65,
        "wind_speed": 5.2
    }
    
    print(f"🌤️  Fetching weather for {city}...")
    print(f"   Temperature: {mock_weather['temperature']}°C")
    print(f"   Conditions: {mock_weather['description']}")
    
    return mock_weather

def get_weather_advice(city: str, budget_manager: TokenBudgetManager = None) -> Dict[str, Any]:
    """
    Fetch weather data and ask LLM for advice.
    
    Args:
        city: City name
        budget_manager: Optional token budget manager
    
    Returns:
        Weather advice from LLM
    """
    # Step 1: Fetch external data
    weather = fetch_weather_data(city)
    
    if not weather:
        return {"error": "Couldn't fetch weather data"}
    
    # Step 2: Create prompt with the data
    prompt = f"""Based on this current weather in {weather['city']}:

Temperature: {weather['temperature']}°C (feels like {weather['feels_like']}°C)
Conditions: {weather['description']}
Humidity: {weather['humidity']}%
Wind speed: {weather['wind_speed']} m/s

Give practical advice for someone going out today. What should they wear? Any precautions?"""
    
    # Step 3: Ask LLM with enriched prompt
    print("\n🤖 Asking LLM for advice...")
    if budget_manager:
        result = ask_openai_with_budget(prompt, budget_manager=budget_manager)
        advice = result['response']
    else:
        advice = ask_openai(prompt)
    
    return {
        "city": city,
        "weather": weather,
        "advice": advice
    }

# Test (commented out)
# result = get_weather_advice("London", budget_manager=budget_manager)
# print(f"\n💡 Advice: {result['advice']}")

---

## 5. Production Configuration

Manage API keys, environments, and settings properly.

In [ ]:
class Config:
    """Configuration for different environments."""
    
    def __init__(self, environment: Optional[str] = None):
        if environment is None:
            environment = os.getenv("ENVIRONMENT", "development")
        
        self.environment = environment
        self.load_config()
    
    def load_config(self):
        """Load configuration based on environment."""
        if self.environment == "production":
            self.debug = False
            self.max_retries = 5
            self.timeout = 60
            self.log_level = "INFO"
            self.use_cache = True
            self.openai_model = "gpt-4o"
            self.daily_budget = float(os.getenv("DAILY_BUDGET", "100.0"))
        
        elif self.environment == "staging":
            self.debug = True
            self.max_retries = 3
            self.timeout = 30
            self.log_level = "DEBUG"
            self.use_cache = True
            self.openai_model = "gpt-4o-mini"
            self.daily_budget = float(os.getenv("DAILY_BUDGET", "10.0"))
        
        else:  # development
            self.debug = True
            self.max_retries = 2
            self.timeout = 10
            self.log_level = "DEBUG"
            self.use_cache = False
            self.openai_model = "gpt-4o-mini"
            self.daily_budget = float(os.getenv("DAILY_BUDGET", "1.0"))
    
    def get_budget_manager(self) -> TokenBudgetManager:
        """Get configured budget manager."""
        return TokenBudgetManager(daily_budget=self.daily_budget)

# Usage
config = Config()
print(f"Environment: {config.environment}")
print(f"Debug mode: {config.debug}")
print(f"Using model: {config.openai_model}")
print(f"Daily budget: ${config.daily_budget:.2f}")

# Get budget manager with correct configuration
budget_manager = config.get_budget_manager()
print("✅ Configuration loaded!")

---

## Summary

You've learned practical patterns for production AI applications:

### Multiple LLM Providers
- ✅ Using OpenAI and Anthropic together
- ✅ Passing outputs between providers
- ✅ Comparing responses from different models

### Provider Fallback
- ✅ Automatic fallback when primary provider fails
- ✅ Cost-based provider selection

### Token Budgets
- ✅ Accurate token counting for different models
- ✅ Budget tracking across providers
- ✅ Cost calculation and budget enforcement
- ✅ Smart routing based on remaining budget

### External Data Enrichment
- ✅ Fetching data from external APIs
- ✅ Injecting data into prompts
- ✅ Real-time information for LLMs

### Production Configuration
- ✅ Environment-based settings
- ✅ Secure API key management
- ✅ Configurable budgets per environment

**Key Takeaways:**

1. **Use multiple providers** - Different models excel at different tasks
2. **Track token budgets** - Prevent cost overruns with budget management
3. **Enrich with data** - Combine LLMs with real-time external data
4. **Plan for failures** - Always have fallback strategies
5. **Configure per environment** - Different settings for dev/staging/prod

**Next Steps:**
- Practice implementing these patterns in your own projects
- Experiment with different budget thresholds
- Try combining multiple data sources
- Monitor token usage in production

## Exercises

Try these exercises to solidify your understanding:

1. **Budget Alerts**: Add alert thresholds (e.g., warn at 50%, 75%, 90% of budget)

2. **Multi-Provider Chain**: Build a pipeline that uses OpenAI for generation, Anthropic for review, and tracks budget for both

3. **External Data Sources**: Integrate a real API (news, weather, stock prices) and enrich prompts

4. **Budget Reset**: Add functionality to reset daily budgets (useful for testing)

5. **Cost Comparison**: Compare costs of using OpenAI vs Anthropic for the same tasks